- try pytorch
- try keras model by nykgoto
- incorporate hyperparameters by optuna
- add concatenation layer in my original keras
- random seed average in normal lightgbm and lightgbm with under-sampling
- ** cosider 511 dimensions of wave df

In [1]:
import cv2
import copy
import time
import math
import tqdm
import random
import pathlib
import warnings
import datetime
import scipy as sp
import numpy as np
import pandas as pd
import scipy.signal
import xgboost as xgb
from time import time
import lightgbm as lgb
from catboost import Pool
from functools import partial
from catboost import CatBoost
from keras.models import Model
from scipy.stats import cauchy
import matplotlib.pyplot as plt
from scipy.stats import laplace
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from keras.layers import Dense, Input
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from scipy.stats import kurtosis, skew
from collections import Counter, defaultdict
from scipy.interpolate import UnivariateSpline
from sklearn.preprocessing import MinMaxScaler
from hyperopt import hp, tpe, Trials, fmin, space_eval
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, average_precision_score, r2_score

np.set_printoptions(precision=5)
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",1000)
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [2]:
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def _1gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x-cen1)/sigma1)**2)))

def _1Lorentzian(x, amp, cen, wid):
    return amp*wid**2/((x-cen)**2+wid**2)

In [3]:
filepath = "../input/atma2020529/"
filepath2 = "../input/atma2020529-2/"
filepath3 = "../input/atma20205293/"
filepath4 = "../input/atam202005294/"
train = pd.read_csv(filepath + "train.csv")
test = pd.read_csv(filepath + "test.csv")
fitting = pd.read_csv(filepath4 + "fitting__fixed.csv")
spec_df = pd.read_csv(filepath3 + "spec.csv")
wave_df = pd.read_csv(filepath2 + "wave_df.csv")
wave_test = pd.read_csv(filepath2 + "wave_test.csv")
wave_df = pd.DataFrame(wave_df)
wave_test = pd.DataFrame(wave_test)
wave_df = wave_df.iloc[:,:511]

In [4]:
wave_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510
0,1751.0,1493.0,1299.0000,1120.0,900.0,820.0,542.2222,458.0,223.0,169.0,262.0,17.0,210.0,430.0,169.0,129.0,164.0,25.0,-80.0,54.0,193.0,65.0,219.0,112.0,178.0,228.0,77.0,127.0,190.0,135.0,189.0,-47.0,73.0,-28.0,26.66667,62.0,210.0,231.0,93.0,3.0,247.0,41.0,97.0,-79.0,114.0,111.0,60.0,102.0,19.0,102.0,17.0,45.0,-3.0,47.0,263.0,248.0,107.0,-70.0,21.0,52.0,-41.0,-27.0,36.0,107.0,147.0,-106.0,-211.0,38.0,-57.0,39.0,84.0000,7.0,92.0,-68.0,25.0,45.0,-16.0,-59.0,-30.0,58.0,-156.0,194.0,18.0,78.0,-132.0,-24.0,26.0,-64.0,-54.0,-45.0,-90.0,-52.0,-15.0,160.0,109.0,-173.0,175.0,-74.0,-51.0,-47.0,215.0,37.0,-47.0,142.0,46.0,216.0,30.0,-98.0,58.0,87.0,59.0,255.000,38.0,104.0,286.0,130.0,245.0,34.0,88.0,195.0,104.0,-65.0,46.0,-43.0,5.0,141.0,151.0,-190.0,34.0,53.0,-27.0,12.0,120.0,75.0,96.0,0.0,-228.0,113.0,65.0,-152.0,147.0,-228.0,-95.0,78.0,-16.0,41.0,94.0,-39.0,63.0,94.0,-36.0,-59.0,22.0,-110.0,-199.0,-14.0,176.0,-12.0,72.0,231.0,-48.0,38.0,5.0,39.0,14.0,9.0,27.0,183.0,40.0,86.0,97.0,103.0,-125.0,-83.0,101.0,13.0000,81.0,-75.0,20.0,21.0,-63.0,-29.0,-50.0,49.0,-8.0,98.0,-15.0,-134.0,4.0,44.00000,15.0,326.0,132.0,-61.0,-65.0,97.0,108.0,85.0,52.0,16.0,-120.0,56.0,-27.0,140.0,9.0,2.0,76.0,205.0,-15.0,-31.0,-62.0,-134.0,175.0,-194.0,-155.0,97.0,-155.0,-10.0,21.0,38.0,-39.0,-39.0,-15.0,-85.0,27.0,2.0,44.0,-104.0,149.0,-5.0,56.00000,-73.0,-13.0,-66.0,61.0,112.0,5.0,-32.0,-99.0,74.0,-49.0,-44.0,-62.0000,-5.0,44.0,-59.0,-149.0,114.0,-117.0,-93.0,243.0,84.0,-109.0,-59.0,-7.0,18.0,-157.0,128.0,-85.0,212.0,-119.0,-52.0,91.0,-32.0,-26.0,109.0,-141.0,-88.0,64.0,100.0,-79.0,155.00000,74.0,-23.0,49.0,37.0000,-25.0,10.0,-84.0,7.0,46.0,-32.0,209.0,-111.0,-123.0,-48.0000,-18.0,-113.0,109.0,18.0,-19.0,49.0,-25.0,0.0,92.0,-105.0,14.0,-28.0,52.0,-69.0,59.0,135.0,39.0,-88.0,129.0000,48.0,-14.0,222.0,117.0,239.0,100.0,8.0,172.2222,209.0,126.0,155.0,199.0,66.0,-72.0,-21.0,-15.0,19.0,40.0,162.0,90.0,-111.0,119.0,-159.0,-71.0,1.0,-34.0,29.0,31.0,303.0,-42.0,18.0,-49.0,-3.0,116.0,105.0,33.0,12.0,62.0,-82.0,34.0000,-149.0,229.0,-1.0,37.0000,-97.0,-116.0,26.0,212.0,-8.0,6

# FE

In [5]:
# 時系列データから特徴量を生成できるtsfresh
#from tsfresh import extract_features, extract_relevant_features
#from tsfresh.feature_extraction import settings
# https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#module-tsfresh.feature_extraction.feature_calculators

# 一気に多数の変数を生成する場合 -----
#ts_df = extract_features(spec_df, column_id="spectrum_filename", column_sort="wavelength", n_jobs=8, 
#                        default_fc_parameters=settings.ComprehensiveFCParameters())
#ts_df = ts_df.head().reset_index()
#ts_df = ts_df.rename(columns={"id":"spectrum_filename"})
# ------------------------------

# 自分で指定する場合 ----
#fc_parameters = {
    #xx"large_standard_deviation": [{ "r": 0.1}],   
    #xx"ar_coefficient": [{"coeff": 3, "k": 10}], 
    #xx"fft_coefficient": [{"coeff": 3, "attr": "real"}],
    #"cid_ce" :[{"normalize": True}],
#    "autocorrelation": [{"lag": 30}],
    #"fft_aggregated" : [{"aggtype": "kurtosis"}],
#}
#ts_df = extract_features(spec_df, column_id="spectrum_filename", column_sort="wavelength", n_jobs=8, 
#                        default_fc_parameters=fc_parameters)
# -------------------

# 一気に生成後、絞り込みを行う場合 ----
#y = spec_df.merge(train, how="left", on="spectrum_filename").set_index("spectrum_filename").target
#y = y.groupby("spectrum_filename").mean()
#ts_df = extract_relevant_features(df, y, column_id="spectrum_filename", column_sort="wavelength")
#ts_df = ts_df.reset_index()
#ts_df = ts_df.rename(columns={"id":"spectrum_filename"})

In [6]:
def peak_near_stats(x):
    i = np.argmax(x)
    y = x[i - 10:i + 10]
    return np.sum(y) / x[i], kurtosis(y)

def peak_near_stats2(x):
    i = np.argsort(-x)[1]
    z = x[i - 3:i + 3]
    return np.sum(z) / x[i]

def peak_near_stats4(x):
    i = np.argsort(-x)[3]
    z = x[i - 3:i + 3]
    return np.sum(z) / x[i]

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# aggragate spec information
spec_agg = spec_df.groupby("spectrum_filename")["intensity"].agg(["max", "min", "mean", "std"])
spec_agg.columns = ["intensity_" + c for c in spec_agg.columns]
fft = []
peak_near_ratio = []
peak_near1 = []
sec_peak_near1 = []
sec_peak_near2 = []
sec_peak_near3 = []

first_dif_loc = []
first = []
second = []
third = []
diff_kurt = []
f_second = []

for i, file_df in tqdm(spec_df.groupby("spectrum_filename")):
    x = file_df["wavelength"].values
    y = file_df["intensity"].values
        
    # fast fourier transformation
    F = np.fft.fft(y)
    Amp = np.abs(F)
    fft.append(np.quantile(Amp, 0.95))
    
    # add peak info
    val1, val2 = peak_near_stats(y)
    peak_near_ratio.append(val1)
    peak_near1.append(val2)
    
    val3 = peak_near_stats2(y)
    sec_peak_near1.append(val3)
    val5 = peak_near_stats4(y)
    sec_peak_near3.append(val5)
    
    # savgol_filter
    smooth = scipy.signal.savgol_filter(y, 5, 2, deriv=0),  # なめらかにしただけ   
    first_deriv = scipy.signal.savgol_filter(y, 5, 2, deriv=1),  # 1次微分
    sec_deriv = scipy.signal.savgol_filter(y, 5, 2, deriv=2),
    
    first_dif_loc.append(np.argmax(first_deriv) - np.argmin(first_deriv))
    #first.append(np.max(first_deriv))
    second.append(np.max(sec_deriv) - np.min(sec_deriv))
    
    # diff
    diff_kurt.append((file_df["intensity"].diff(2).diff() + file_df["intensity"].diff(2).diff(-1)).kurt())

spec_agg["amp_0.95"] = fft
spec_agg["peak_ratio"] = peak_near_ratio
spec_agg["peak_near_kur"] = peak_near1
spec_agg["sec_peak_ratio"] = sec_peak_near1
spec_agg["fou_peak_ratio"] = sec_peak_near3

#spec_agg["first_deriv_diff"] = first
spec_agg["sec_deriv_diff"] = second
spec_agg["first_deriv_loc"] = first_dif_loc
#spec_agg["third_deriv_diff"] = third

spec_agg["diff_kurt"] = diff_kurt

In [7]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.abs(nodes - node)
    return np.argmin(dist_2)

def new_peak_near_stats(x, idx):
    y = x[idx - 3:idx + 3]
    return np.sum(y) / x[idx], kurtosis(y)

def transform(df):
    new_df = df.copy()
    # merge original csv and fitting data
    new_df = pd.merge(new_df, fitting, on="spectrum_id", how="left")
    
    # merge original csv and spec information
    new_df= pd.merge(new_df, spec_agg.reset_index(), on="spectrum_filename", how="left")
    
    #new_df = pd.merge(new_df, ts_df, on="spectrum_filename", how="left")
    
    # remove unnecessary columns
    if "target" in new_df.columns:
        new_df = new_df.drop(["spectrum_id", "spectrum_filename", "layout_x","layout_y", "pos_x", 'layout_a'], axis=1)
    else:
        new_df = new_df.drop(["spectrum_id","chip_id","spectrum_filename", "layout_x", "layout_y", "pos_x",'layout_a'], axis=1)
    
    # create new variables
    new_df["ratio2_5"] = new_df["params2"] / new_df["params5"] # I don't know the meaning, but seems effective
    new_df["ratio3_1"] = new_df["params3"] / new_df["params1"] # I don't know the meaning, but seems effective
    new_df["ratio4_6"] = new_df["params4"] / new_df["params6"] # I don't know the meaning, but seems effective
    
    return new_df
new_train = transform(train)
new_test = transform(test)
print(f'train shape: {new_train.shape}')
print(f'test shape: {new_test.shape}')

train shape: (7436, 27)
test shape: (6952, 25)


In [8]:
new_train.head()

,chip_id,exc_wl,target,params0,params1,params2,params3,params4,params5,params6,rms,beta,intensity_max,intensity_min,intensity_mean,intensity_std,amp_0.95,peak_ratio,peak_near_kur,sec_peak_ratio,fou_peak_ratio,sec_deriv_diff,first_deriv_loc,diff_kurt,ratio2_5,ratio3_1,ratio4_6
0,79ad4647da6de6425abf,850,0,30.808589,5.811802e+02,1037.714752,1.531423,22469.651641,1032.317268,8.295610,10.028668,2.521298e-02,1751.0,-228.0,40.292752,172.206792,7643.154333,0.000000,NaN,0.000000,6.591964,302.428571,189,0.034112,1.005229,2.635022e-03,2708.619691
1,79ad4647da6de6425abf,780,0,91.300897,1.740582e+04,1080.510452,4.766233,33257.123175,1077.468855,8.018225,7.948485,3.435612e-01,4219.0,-263.0,166.958984,463.428363,28918.587810,9.569329,-1.257037,5.422615,5.596097,578.285714,-6,0.103895,1.002823,2.738298e-04,4147.691287
2,c695a1e61e002b34e556,780,0,106.642946,1.000000e-10,1119.464438,2.000000,42579.867913,1378.883338,11.687417,10.739859,2.348528e-15,2412.0,-235.0,151.577691,327.857694,18130.782140,12.473051,-1.282283,5.644407,5.754322,337.285714,-7,-0.259870,0.811863,2.000000e+10,3643.223069
3,c695a1e61e002b34e556,780,0,306.933674,1.099486e+04,1139.855067,5.198692,39349.741703,1145.212849,9.445029,10.379948,2.183921e-01,3209.0,-52.0,523.080947,436.481410,22149.537147,12.990825,-1.479866,5.701270,5.879316,373.857143,-10,-0.255881,0.995322,4.728291e-04,4166.185449
4,c695a1e61e002b34e556,780,0,46.133256,2.227622e+04,1120.918337,5.668012,31054.928673,1117.107782,7.658710,8.316550,4.176962e-01,3998.0,-245.0,138.187717,472.009931,30874.913315,10.321911,-1.386428,5.176440,5.750939,491.714286,-7,-0.125171,1.003411,2.544423e-04,4054.851290


# keras model by nykgoto

In [9]:
## CNN
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Model
from keras.layers import Dense, Input, BatchNormalization, ReLU, Flatten
from keras.layers import Reshape, Conv1D, MaxPool1D, Input, BatchNormalization, ReLU, Flatten, Dense, GlobalMaxPool1D
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
from keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight

################preprocess#####################
print('train before: {}'.format(wave_df.shape))
x_train = np.array(wave_df)[:, :, None, None]
print('train after: {}'.format(x_train.shape))
x_test = np.array(wave_test)[:, :, None, None]

################augmentation###################
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 学習時に使う generator. 
# featurewise_std_normalization でデータセット全体で正規化 + height (i.e. 511次元の方向) にちょっとずらす shift を行なう
# vertical_flip(波形方向に反転)は試したのですがしないほうがCVが良かったため採用していません
gen = ImageDataGenerator(vertical_flip=False,  
                         featurewise_std_normalization=True,
                         height_shift_range=.05)

# validation の時に使う generator
# こちらは validation に使うのでデータの変更は行わない 
# (ただし正規化のスケールは合わせないと train とずれるので featurewise の正規化だけかける) 
valid_gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    vertical_flip=False, 
    height_shift_range=0
)

gen.fit(x_train)
valid_gen.fit(x_train)

###############modelling#####################
def conv_layer(x, n_channels=32, kernel_size=3, pool_size=2, padding='same'):
    x = Conv1D(filters=n_channels, kernel_size=kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    return x

def conv1d_network(n_inputs=511, n_output=1):
    input_tensor = Input(shape=(n_inputs, 1, 1))
    x = Reshape((n_inputs, 1))(input_tensor)
    x = conv_layer(x, 8, 5, 2)
    x = conv_layer(x, 32, 5, 2)
    x = conv_layer(x, 64, 5, 2)
    x = conv_layer(x, 128)
    # 今回単純化のためにこれより下はコメントアウトしています。GPU ON の時はコメントアウト外して実行してみてください
    x = conv_layer(x, 256) #
    x = conv_layer(x, 512) #
    x = conv_layer(x, 1024) #
    x = conv_layer(x, 2048) #
    x = GlobalMaxPool1D()(x)
    out = Dense(n_output, activation='sigmoid')(x)
    return Model(input_tensor, outputs=out, name='conv1d')

class PRAUCCallback(Callback):
    def __init__(self, valid_flow, y):
        super(PRAUCCallback, self).__init__()
        self.valid_flow = valid_flow
        self.y = y

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict_generator(valid_flow).reshape(-1)
        score = average_precision_score(self.y, y_pred)
        print(f'PR AUC: {score:4f}')
        logs['val_pr-auc'] = score
        return

    def on_batch_begin(self, batch, logs=None):
        return

    def on_batch_end(self, batch, logs=None):
        return
    
def get_lr(epoch):
    if epoch < 20:
        return 1e-3

    return 1e-4

def get_cv_byname(name='stratified'):
    if name == 'stratified':
        fold = StratifiedKFold(n_splits=5, random_state=71, shuffle=True)
        return list(fold.split(x_train, y))

    if name == 'group':
        fold = GroupKFold(n_splits=5)
        cv = list(fold.split(x_train, y, train['chip_id']))
        return cv
    
model_conv = conv1d_network(511, 1)
print(model_conv.summary())

# inbalance の対応としてクラスの重みを計算して, positive に大きな重みをもたせる
class_weight = compute_class_weight('balanced', np.unique(train["target"]), train["target"])
print('class weight: {}'.format(class_weight))
    
y = train["target"].values

oof = np.zeros_like(y).astype(float)
n_batch = 128
cv_name = 'group'
use_best_model = True
models = []

print(f'CV Strategy: {cv_name}')
print(f'Use Best Model {use_best_model}')

cv = get_cv_byname(cv_name)

# to keep reproducible results ----------------
# https://keras.io/getting_started/faq/
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)
tf.random.set_seed(1234)
# ------------------------------------------

for i, (idx_tra, idx_val) in enumerate(cv):
    print(f'start FOLD={i + 1}')

    train_flow = gen.flow(x_train[idx_tra], y[idx_tra], batch_size=n_batch)
    valid_flow = valid_gen.flow(x_train[idx_val], y[idx_val], batch_size=n_batch, shuffle=False)

    model_i = conv1d_network()
    model_i.compile(loss='binary_crossentropy', optimizer='adam')

    model_i.fit(train_flow, 
                steps_per_epoch=len(x_train) / n_batch, 
                validation_data=valid_flow,
                epochs=30, 
                class_weight=class_weight,
                callbacks=[
                    PRAUCCallback(valid_flow, y=y[idx_val]),
                    ModelCheckpoint(filepath=f'./fold_{i}.hdf5', verbose=1, mode='max',
                                    save_best_only=True, monitor='val_pr-auc'),
                    LearningRateScheduler(get_lr)
                ])

    if use_best_model:
        model_i = load_model(f'./fold_{i}.hdf5')
    pred_i = model_i.predict_generator(valid_flow).reshape(-1)
    oof[idx_val] = pred_i
    models.append(model_i)
    score = average_precision_score(y[idx_val], pred_i)
    print(f'Current Fold PR-AUC: {score:.4f}')

train before: (7436, 511)
train after: (7436, 511, 1, 1)
Model: "conv1d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 511, 1, 1)         0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 511, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 511, 8)            48        
_________________________________________________________________
batch_normalization_1 (Batch (None, 511, 8)            32        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 511, 8)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 256, 8)            0         
_________________________________________________________________
con

In [10]:
scores = []
for _, idx_val in cv:
    scores.append(average_precision_score(y[idx_val], oof[idx_val]))

print('=' * 20 + 'Out-Of-Fold Score' + '=' * 20)
print(np.mean(scores), np.std(scores))

test_flow = valid_gen.flow(x_test, shuffle=False, batch_size=128)

pred = []
for model in models:
    pred.append(model.predict_generator(test_flow))
nyk_pred = np.mean(pred, axis=0)[:, 0]
nyk_score = np.mean(scores)

====================Out-Of-Fold Score====================
0.8893968732294084 0.0517628862721894


# keras model by my effort

In [11]:
## CNN
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight

## 次元圧縮
import umap
from scipy.sparse.csgraph import connected_components

# https://teratail.com/questions/170548 を参考に交互に属性を複数チャネル形式に変換する
wave_500 = (np.array(wave_df) * (np.array(wave_df) > 500))
i = range(wave_df.shape[1]) #追加元の配列
wave_tr = np.insert(wave_500,i,np.array(wave_df)[:,i],axis=1).reshape(-1,511,2)

wave_500_test = (np.array(wave_test) * (np.array(wave_test) > 500))
i = range(wave_test.shape[1]) #追加元の配列
wave_te = np.insert(wave_500_test,i,np.array(wave_test)[:,i],axis=1).reshape(-1,511,2)
# ----------------------------------------

nn_y_train = new_train.target.copy()
nn_X_train = np.array(wave_tr).reshape(-1, 511, 2) #axis=0がデータid, axis=1:データ数, axis=2:ch数
nn_X_test = np.array(wave_te).reshape(-1, 511, 2)
nn_y_train = np.array(nn_y_train).reshape(-1, 1)

class_weight = compute_class_weight('balanced', np.unique(new_train.target.copy()), new_train.target.copy())

# concatenateで追加する ------
exc_wl = np.array(new_train[["exc_wl",]].copy()) #"exc_wl",
exc_wl_test = np.array(new_test[[ "exc_wl"]].copy()) #"exc_wl",
# --------------------------

def sk_pr_auc(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

def get_lr(epoch): # change learning rate by epoch
    if epoch < 20:
        return 1e-3

    return 1e-4

def create_model(input_len, n_filter, filter_size, drop_rate, gauss_rate):

    ## 入力の型を定義
    _input = Input((input_len, 2))
    _input_aux = Input((1,))
    
    x = GaussianNoise(gauss_rate)(_input)

    ## 畳み込みブロック(過学習防止にdropoutも追加します)
    x = Conv1D(n_filter, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)

    ## 2層目
    x = Conv1D(n_filter, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)
    
    ## 3層目
    x = Conv1D(n_filter, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)

    ## あとで挙動を確認したいので、この層に名前をつけます
    x = GlobalMaxPool1D(name="hidden")(x)
    
    x = Concatenate()([x, _input_aux])
    x = Dense(64)(x)
    x = Activation("relu")(x)

    x = Dropout(drop_rate)(x)

    ## 0 or 1を当てるのでsigmoidを使います
    _out = Dense(1, activation="sigmoid")(x)

    #model = Model(_input,_out, name="Sequential")
    model = Model([_input, _input_aux],_out, name="Sequential")

    return model

def extra_dist(best_model, tr, va, te):

    ## 中間層を出力するためのモデル
    layer_name = 'hidden'
    hidden_model = Model(inputs=best_model.input,
                         outputs=best_model.get_layer(layer_name).output)


    ## trainを入力し、64次元のベクトル抽出
    hidden_pred_train = hidden_model.predict(tr)
    hidden_pred_valid = hidden_model.predict(va)
    hidden_pred_test = hidden_model.predict(te)

    ## UMAPによる次元圧縮
    trans = umap.UMAP(n_components=2, random_state=42)
    train_dist = trans.fit(hidden_pred_train)
    valid_dist = trans.transform(hidden_pred_valid)
    test_dist = trans.transform(hidden_pred_test)

    return train_dist, valid_dist, test_dist

def modelling_nn(tr, target, te):
    
    # to keep reproducible results ----------------
    # https://keras.io/getting_started/faq/
    import os
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(42)
    random.seed(12345)
    tf.random.set_seed(1234)
    # ------------------------------------------

    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    models = []

    oof = np.zeros(X_train.shape[0])
    feat0 = np.zeros(X_train.shape[0]) # for extracting intermediate vars
    feat1 = np.zeros(X_train.shape[0]) # for extracting intermediate vars
    test_feat0 = np.zeros(X_test.shape[0]) # for extracting intermediate vars
    test_feat1 = np.zeros(X_test.shape[0]) # for extracting intermediate vars
    scores = []
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train[train_index,:]
        y_train2 = y_train[train_index]
        exc_wl_train = exc_wl[train_index]

        X_test2 = X_train[test_index,:]
        y_test2 = y_train[test_index]
        exc_wl_valid = exc_wl[test_index]
        
        clf = create_model(input_len=511, n_filter=64, filter_size=7, drop_rate=0.2, gauss_rate = 0.07)
        clf.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=[AUC(curve='PR', num_thresholds=1000)])
        
        ## 学習, bestmodelをsave
        ckp = ModelCheckpoint(filepath='best_weight_cv_{}.h5'.format(i+1), 
                          verbose=1, save_best_only=True)
        history = clf.fit([X_train2, exc_wl_train],y_train2,
                        validation_data=([X_test2,exc_wl_valid], y_test2),
                        class_weight=class_weight,
                        epochs=30,
                        batch_size=16,
                        callbacks = [ckp, LearningRateScheduler(get_lr)],
                        verbose=1)
        
        ## 中間ベクトルを取り出すためのモデル
        pred_model = create_model(input_len=511, n_filter=64, filter_size=7, drop_rate=0.2, gauss_rate = 0.07)
        pred_model.load_weights('best_weight_cv_{}.h5'.format(i+1))

        ## extracing intermediate vectors as new features
        #tmp0 = np.zeros(X_train.shape[0]) # for extracting intermediate vars
        #tmp1 = np.zeros(X_train.shape[0])
        #tmp0_test = np.zeros(X_test.shape[0])
        #tmp1_test = np.zeros(X_test.shape[0])
        #train_dist, valid_dist, test_dist = extra_dist(pred_model, X_train2, X_test2, X_test)
        #tmp0[train_index] = np.array(train_dist.embedding_[:,0])
        #tmp0[test_index] = np.array(valid_dist[:,0])
        #tmp1[train_index] = np.array(train_dist.embedding_[:,1])
        #tmp1[test_index] = np.array(valid_dist[:,1])
        #tmp0_test = np.array(test_dist[:,0])
        #tmp1_test = np.array(test_dist[:,1])

        #feat0 += tmp0 / n_folds
        #feat1 += tmp1 / n_folds
        #test_feat0 += tmp0_test / n_folds
        #test_feat1 += tmp1_test / n_folds
        
        ## intermediate vectors→dimension reduction→visualization
        #plt.figure(figsize=(8,4), tight_layout=True)
        #plt.subplot(1,2,1)
        #plt.scatter(train_dist.embedding_[:,0], train_dist.embedding_[:,1],
        #        c=y_train2[:, 0],
        #        cmap="bwr",
        #        alpha=0.3,
        #        s=10)
        #plt.title("Train distribution CV={}".format(i+1))
        #plt.colorbar()

        #plt.subplot(1,2,2)
        #plt.scatter(test_dist[:,0], test_dist[:,1],
        #        c=y_test2[:, 0],
        #        cmap="bwr",
        #        alpha=0.3,
        #        s=10)
        #plt.title("Valid distribution CV={}".format(i+1))
        #plt.colorbar()
        #plt.savefig("CV_{}_dist.png".format(i+1))
        #plt.close()
        
        val_pred = pred_model.predict([X_test2,exc_wl_valid])[:,0]
        oof[test_index] = val_pred
        scores.append(average_precision_score(y_test2, val_pred))
        models.append(pred_model)
    
    score = average_precision_score(target, oof)
    print("average precision score = {}".format(score))
    print("scores in folds:", scores)
    print("mean in folds:", np.mean(scores))
    print("std in folds:", np.std(scores))
    print(confusion_matrix(target, np.round(oof)))
    pred_value = np.zeros(X_test.shape[0])
    for i, model in enumerate(models):
        pred_value += model.predict([X_test,exc_wl_test])[:,0] / len(models)
    return score, pred_value, oof, feat0, feat1, test_feat0, test_feat1

#metric_nn, pred_value_nn, oof_nn, tr_feat0, tr_feat1, te_feat0, te_feat1 = modelling_nn(nn_X_train, nn_y_train, nn_X_test)

#from IPython.display import Image
#file_name = "CV_5_dist.png"
#Image(file_name)

#mod_train = new_train.copy()
#mod_train["nn1"] = tr_feat0
#mod_train["nn2"] = tr_feat1
#mod_test = new_test.copy()
#mod_test["nn1"] = te_feat0
#mod_test["nn2"] = te_feat1

# Pytorch model by my effort

In [12]:
# custom metric
# keras conv1dとpytorch conv1dの比しないとダメ

## CNN pytorch version
# kaggle titanic pytorch: https://www.kaggle.com/kaerunantoka/titanic-pytorch-nn-tutorial
# https://github.com/pytorch/pytorch/issues/3867#issuecomment-598264120 (conv1d same padding)
# https://stackoverflow.com/questions/42480111/model-summary-in-pytorch (model summary in pytorch)
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from sklearn.utils.class_weight import compute_class_weight

batch_size = 16
train_epochs = 50

# https://teratail.com/questions/170548: 交互に属性を複数チャネル形式に変換する
wave_500 = (np.array(wave_df) * (np.array(wave_df) > 500))
i = range(wave_df.shape[1]) #追加元の配列
wave_tr = np.insert(wave_500,i,np.array(wave_df)[:,i],axis=1).reshape(-1,2,511)

wave_500_test = (np.array(wave_test) * (np.array(wave_test) > 500))
i = range(wave_test.shape[1]) #追加元の配列
wave_te = np.insert(wave_500_test,i,np.array(wave_test)[:,i],axis=1).reshape(-1,2,511)
# ----------------------------------------

nn_y_train = new_train.target.values
nn_X_train = np.array(wave_tr).reshape(-1, 2, 511) # axis=0がデータid, axis=1:ch数, axis=2:データ数
nn_X_test = np.array(wave_te).reshape(-1, 2, 511)

# https://stackoverflow.com/questions/57021620/how-to-calculate-unbalanced-weights-for-bcewithlogitsloss-in-pytorch (class weight)
weights = compute_class_weight('balanced', np.unique(new_train.target.copy()), new_train.target.copy())
class_weight = torch.tensor(weights[1] / weights[0], dtype=torch.float32)

exc_wl = np.array(new_train[["exc_wl"]].copy()) 
exc_wl_test = np.array(new_test[["exc_wl"]].copy()) 

# to  keep reproducibility -----
def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
#sample_seed = 1337

def sk_pr_auc(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Net(nn.Module):
    def __init__(self, n_filter, filter_size, drop_rate, gauss_rate):
        super(Net, self).__init__()
        self.gauss_rate = gauss_rate
        self.conv1 = nn.Conv1d(2, 2, n_filter, stride = 1, padding=n_filter//2) # same padding
        self.batch1 = nn.BatchNorm1d(2)
        self.drop1 = nn.Dropout(drop_rate)
        
        self.conv2 = nn.Conv1d(2, 2, n_filter, stride = 1, padding=n_filter//2-1)
        self.batch2 = nn.BatchNorm1d(2)
        self.drop2 = nn.Dropout(drop_rate)
        
        self.conv3 = nn.Conv1d(2, 2, n_filter, stride = 1, padding=n_filter//2)
        self.batch3 = nn.BatchNorm1d(2)
        self.drop3 = nn.Dropout(drop_rate)
        
        self.drop5 = nn.Dropout(drop_rate)

        self.lr2 = nn.Linear(64,1)

    def forward(self, x):#, x_aux):
        #https://stackoverflow.com/questions/59090533/how-do-i-add-some-gaussian-noise-to-a-tensor-in-pytorch GaussianNoise(gauss_rate)
        x = x + self.gauss_rate * torch.randn(x.shape[0], 2, 511) 
        #print("1:", x.shape)
        x = self.drop1(self.batch1(F.relu(self.conv1(x))))
        #print("2:", x.shape)
        x = self.drop2(self.batch2(F.relu(self.conv2(x))))
        #print("3:", x.shape)
        x = self.drop3(self.batch3(F.relu(self.conv3(x))))
        #print("4:", x.shape)
        
        # https://discuss.pytorch.org/t/global-max-pooling/1345 (x = GlobalMaxPool1D(x))
        x, _ = torch.max(x, 1)
        
        # https://discuss.pytorch.org/t/concatenate-layer-output-with-additional-input-data/20462 (x = Concatenate()([x, x_aux]))
        #x = torch.cat((x, x_aux), dim=1)
        
        x = x.view(x.size(0), -1) # https://discuss.pytorch.org/t/tensor-reshaping/16874 (reshape)
        x = self.drop5(F.relu(nn.Linear(x.size()[-1],64)(x)))
        _out = self.lr2(x)
        return _out

def modelling_torch(tr, target, te, exc_wl, exc_wl_test,sample_seed):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    exc_wl_test = torch.tensor(exc_wl_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) #, exc_wl_test
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)

    oof = np.array([])
    real = np.array([])
    pred_value = np.zeros(te.shape[0])
    scores = []
    for i , (train_index, valid_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        # https://stackoverflow.com/questions/56741087/how-to-fix-runtimeerror-expected-object-of-scalar-type-float-but-got-scalar-typ
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index, np.newaxis], dtype=torch.float32)
        exc_wl_train = torch.tensor(exc_wl[train_index], dtype=torch.float32)        

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index, np.newaxis], dtype=torch.float32)
        exc_wl_valid = torch.tensor(exc_wl[valid_index], dtype=torch.float32)
                
        clf = Net(n_filter=64, filter_size=7, drop_rate=0.2, gauss_rate = 0.07)
        #print(clf)
        loss_fn = torch.nn.BCEWithLogitsLoss(reduction="mean", weight=class_weight)
        optimizer = optim.Adam(clf.parameters(), lr=0.001)
        # http://katsura-jp.hatenablog.com/entry/2019/01/30/183501#%E8%87%AA%E4%BD%9Cscheduler
        # https://stackoverflow.com/questions/60050586/pytorch-change-the-learning-rate-based-on-number-of-epochs
        #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # https://discuss.pytorch.org/t/make-a-tensordataset-and-dataloader-with-multiple-inputs-parameters/26605
        train = torch.utils.data.TensorDataset(X_train2, y_train2) #, exc_wl_train # can take arbitrary number of inputs
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2) #, exc_wl_valid
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True): #exc_batch, 
                y_pred = clf(x_batch) #, exc_batch
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)        
            
            clf.eval()
            valid = np.zeros((X_valid2.size(0)))
            target_fold = np.zeros((X_valid2.size(0)))
            test_preds_fold = np.zeros(len(X_test))
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader): #exc_batch, 
                y_pred = clf(x_batch).detach() #, exc_batch
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                valid[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
                target_fold[i * batch_size:(i+1) * batch_size] = y_batch.cpu().numpy().reshape(-1,)
        
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
                epoch + 1, train_epochs, avg_loss, avg_val_loss, elapsed_time))
            #scheduler.step()
            #print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
        
        oof = np.concatenate([oof, valid])
        real = np.concatenate([real, target_fold])
        
        #clf.compile(loss='binary_crossentropy',
        #      optimizer='adam', 
        #      metrics=[AUC(curve='PR', num_thresholds=1000)])
        
        # https://discuss.pytorch.org/t/how-to-save-the-best-model/84608 (save best model)
        torch.save(clf.state_dict(), 'best-model-parameters.pt')
        pred_model = Net(n_filter= 64, filter_size=7, drop_rate=0.2, gauss_rate = 0.07)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))
        
        # test predcition --------------
        for i, (x_batch,) in enumerate(test_loader): #, exc_batch
            y_pred = pred_model(x_batch).detach() #, exc_batch
            test_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
        pred_value += test_preds_fold / n_folds
        # ------------------------------
        
        print(average_precision_score(target_fold, valid))
        scores.append(average_precision_score(target_fold, valid))

    score = average_precision_score(real, oof)
    print("average precision score = {}".format(score))
    print("scores in folds:", scores)
    print("mean in folds:", np.mean(scores))
    print("std in folds:", np.std(scores))
    print(confusion_matrix(real, np.round(oof)))
    
    return score, pred_value, oof

#metric_nn, pred_value_nn, oof_nn = modelling_torch(nn_X_train, nn_y_train, nn_X_test, exc_wl, exc_wl_test, sample_seed = 13)

# hypara search

In [13]:
import optuna.integration.lightgbm as lgb ###### optuna ######
import json

categoricals = []
def modelling_optuna(new_train):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)

    valid = np.array([])
    real = np.array([])
    mean_score = 0
    best_params_list = []
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        best_params, tuning_history = dict(), list()
        lgbm_params = {'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'tree_learner': 'serial'}
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 0, feval=pr_auc_metric, categorical_feature = categoricals,
                    best_params=best_params, tuning_history=tuning_history) 
            
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        mean_score += average_precision_score(y_test2,valid_predict) / n_folds
        valid = np.concatenate([valid, valid_predict])
        real = np.concatenate([real, y_test2])
        
        #pd.DataFrame(tuning_history).to_csv('./tuning_history.csv')
        best_params_list.append(best_params)
    
    #for j in range(n_folds):
    #    print('Fold: ' + str(j+1) + ' Best parameters: ' + json.dumps(best_params_list[j], indent=4))

    #print('Best parameters: ' + json.dumps(best_params, indent=4))

    score = average_precision_score(real, valid)
    print("mean score = {}".format(mean_score))
    print("average precision score = {}".format(score))
    return best_params_list

#best_params_list = modelling_optuna(new_train)
#best_params_list

In [14]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': 'None', 
        'objective': 'binary', 
        "tree_learner": "serial",
        'max_depth': int(params['max_depth']),
        'bagging_freq': int(params['bagging_freq']),
        'bagging_fraction': float(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'feature_fraction': float(params['feature_fraction']),
        'learning_rate': float(params['learning_rate']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
}
        X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]
        real = np.array([])
        pred = np.array([])
        skf = StratifiedKFold(n_splits=5)
        for trn_idx, val_idx in skf.split(X, Y):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = ['layout_a'])
            val_set = lgb.Dataset(x_val, y_val, categorical_feature = ['layout_a'])
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 50, 
                         valid_sets = [train_set, val_set], feval=pr_auc_metric, verbose_eval = 1000)
            pred = np.concatenate((pred, np.array(clf.predict(x_val, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val)), axis=0) 
        score = average_precision_score(real, pred)
    
        return - score

    trials = Trials()

    space ={
        'max_depth': hp.quniform('max_depth', 1, 5, 1),
        'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.2, 1.0),
        'num_leaves': hp.quniform('num_leaves', 5, 30, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.2, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0.01, 1.0),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 8, 128, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 30, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=50, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#Y = new_train.target.copy()
#X = new_train.drop(["target"], axis=1).copy()
#random_state = 42
#lbl = preprocessing.LabelEncoder()
#lbl.fit(list(X["chip_id"]))
#X["chip_id"] = lbl.transform(list(X["chip_id"]))
#my_hyperopt(X, Y)

# LightGBM

In [15]:
import lightgbm as lgb
categoricals = []

def modelling_skf(new_train, new_test):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    models = []

    valid = np.array([])
    real = np.array([])
    features_list = [i for i in X_train.columns if i != "chip_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    scores = []
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 
                       'lambda_l1': 0.0,
                        'lambda_l2': 0.0,
                        'num_leaves': 31,
                      'feature_fraction': 1.0,
                      'bagging_fraction': 0.8089800437020872,
                      'bagging_freq': 4,
                      'min_child_samples': 20, 'seed':40+i, 'learning_rate': 0.03, "num_leaves": 10, 'max_depth': 5}

        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
            
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        scores.append(average_precision_score(y_test2,valid_predict))
        valid = np.concatenate([valid, valid_predict])
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    score = average_precision_score(real, valid)
    print("mean score = {}".format(np.mean(scores)))
    print("std score = {}".format(np.std(scores)))
    print("average precision score = {}".format(score))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return score, pred_value, feature_importance_df

metric_skf, pred_value_skf, feature_importance_df_skf = modelling_skf(new_train, new_test)

Fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[404]	training's pr_auc: 1	valid_1's pr_auc: 0.939434
Fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[267]	training's pr_auc: 1	valid_1's pr_auc: 0.913319
Fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[193]	training's pr_auc: 0.999413	valid_1's pr_auc: 0.899655
Fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[253]	training's pr_auc: 1	valid_1's pr_auc: 0.812427
Fold 5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[510]	training's pr_auc: 1	valid_1's pr_auc: 0.942602
mean score = 0.901487595507707
std score = 0.047340088670386096
average precision score = 0.9003618367258028


In [16]:
feature_importance_df_skf.sort_values("Average", ascending=False).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,sec_deriv_diff,274,236,227,221,325,256.6,38.856660,0.151429
1,rms,261,131,101,114,309,183.2,85.027995,0.464127
2,diff_kurt,204,121,119,156,275,175.0,58.743510,0.335677
3,peak_near_kur,200,125,73,138,220,151.2,53.086345,0.351100
4,fou_peak_ratio,196,143,71,93,242,149.0,63.425547,0.425675
5,params0,207,131,67,116,206,145.4,54.194465,0.372727
6,ratio2_5,179,93,70,89,225,131.2,60.114557,0.458190
7,first_deriv_loc,152,104,75,107,215,130.6,48.861437,0.374130
8,ratio3_1,165,101,69,139,169,128.6,38.416663,0.298730
9,intensity_mean,123,140,73,89,205,126.0,46.095553,0.365838


# LightGBM with under-sampling ensembling

In [17]:
from imblearn.under_sampling import RandomUnderSampler
categoricals = []

def modelling_with_us(new_train, new_test):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    models = []
    cv_mean = []
    oof_mean = []
    random_try = 20
    for try_time in range(random_try):
        print("random under sampling:", try_time)
        valid = np.array([])
        real = np.array([])
        features_list = [i for i in X_train.columns if i != "chip_id"]
        feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        mean_score = 0
        for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
            lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 
                       'lambda_l1': 0.0,
                        'lambda_l2': 0.0,
                        'num_leaves': 31,
                      'feature_fraction': 1.0,
                      'bagging_fraction': 0.8089800437020872,
                      'bagging_freq': 4,
                      'min_child_samples': 20, 'seed':40+i, 'learning_rate': 0.03, "num_leaves": 10, 'max_depth': 5}
            print("Fold "+str(i+1))
            X_train2 = X_train.iloc[train_index,:]
            y_train2 = y_train.iloc[train_index]

            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]      
        
            X_train2.drop(["chip_id"], axis=1, inplace=True)
            X_test2.drop(["chip_id"], axis=1, inplace=True)
        
            rus = RandomUnderSampler(sampling_strategy = {0:sum(y_train2==0)//2, 1:sum(y_train2==1)}, random_state=try_time)
            X_resampled, y_resampled = rus.fit_resample(X_train2, y_train2)  
            X_train2 = pd.DataFrame(X_resampled, columns = X_train2.columns)
            y_train2 = pd.DataFrame(y_resampled, columns = ["target"])
        
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
               num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
            
            valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
            mean_score += average_precision_score(y_test2,valid_predict) / n_folds
            valid = np.concatenate([valid, valid_predict])
            real = np.concatenate([real, y_test2])
            feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
            models.append(clf)
        
        feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

        score = average_precision_score(real, valid)
        print("mean score = {}".format(mean_score))
        print("average precision score = {}".format(score))
        cv_mean.append(mean_score)
        oof_mean.append(score)
              
    print(len(models))
    print("between CV score: ", np.mean(cv_mean), np.std(cv_mean))
    print("between OOF score: ", np.mean(oof_mean), np.std(oof_mean))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / (len(models) * random_try)
    return score, pred_value, feature_importance_df

metric_lgb_ens, pred_value_lgb_ens, feature_importance_df_lgb_ens = modelling_with_us(new_train, new_test)

random under sampling: 0
Fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[168]	training's pr_auc: 0.999022	valid_1's pr_auc: 0.958696
Fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[178]	training's pr_auc: 0.999479	valid_1's pr_auc: 0.889636
Fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[327]	training's pr_auc: 1	valid_1's pr_auc: 0.893094
Fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[318]	training's pr_auc: 1	valid_1's pr_auc: 0.824087
Fold 5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[262]	training's pr_auc: 1	valid_1's pr_auc: 0.928595
mean score = 0.898821848469161
average precision score = 0.8855320124148092
random under sampling: 1
Fold 1
Training until validation scores don't improve for 100 rounds
E

# submission

In [18]:
# simple lgb
lgb_sub = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
lgb_sub["target"] = pred_value_skf
lgb_sub.to_csv("atmacup3_simple_lgb_sub"+str(metric_skf)[:-10]+".csv", index = False)
lgb_sub.head()

,target
0,0.000087
1,0.000134
2,0.000068
3,0.000063
4,0.008520


In [19]:
# simple nn
nn_sub = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
nn_sub["target"] = nyk_pred
nn_sub.to_csv("atmacup3_simple_nn_sub"+str(nyk_score)[:-10]+".csv", index = False)
nn_sub.head()

,target
0,0.002621
1,0.000709
2,0.000191
3,0.000671
4,0.001871


In [20]:
# lgb ensemble by random under sampling
lgb_ens_sub = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
lgb_ens_sub["target"] = pred_value_lgb_ens
lgb_ens_sub.to_csv("atmacup3_lgb_us_ens_su"+str(metric_lgb_ens)[:-10]+".csv", index = False)
lgb_ens_sub.head()

,target
0,0.000029
1,0.000038
2,0.000025
3,0.000024
4,0.001039


In [21]:
# nn and (lgb ensemble) ensemble
ens_sub = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
ens_sub["target"] = nyk_pred * 0.3 + pred_value_lgb_ens * 0.7
ens_sub.to_csv("atmacup3_ens_nn_lgbens_sub.csv", index = False)
ens_sub.head()

,target
0,0.000807
1,0.000239
2,0.000075
3,0.000218
4,0.001289


In [22]:
# https://www.kaggle.com/c/homesite-quote-conversion/discussion/18067
# rank average 1
#from scipy.stats import rankdata
#avg_sub = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
#lgb_rank = np.array(rankdata(lgb_sub.target)).reshape(-1,1) # make raking of lgb prediction
#nn_rank = np.array(rankdata(nn_sub.target)).reshape(-1,1) # make raking of nn prediction
#ranks = np.concatenate([lgb_rank,nn_rank], axis=1)
#rank_average = ranks.mean(1)
#avg_sub['target'] = MinMaxScaler().fit_transform(rank_average.reshape(-1, 1))

#avg_sub.to_csv("atmacup3_sample_submission_avg.csv", index = False)
#avg_sub.head()

In [23]:
# rank average 2
# https://www.kaggle.com/kosiew/rank-averaging-script
#avg_sub2 = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
#lgb_rank = np.array(lgb_sub.target.rank()).reshape(-1,1) # make raking of lgb prediction
#nn_rank = np.array(nn_sub.target.rank()).reshape(-1,1) # make raking of nn prediction
#ranks = np.concatenate([lgb_rank,nn_rank], axis=1)
#rank_sum = ranks.sum(1)
#avg_sub2['target'] = rank_sum /(2 * avg_sub2.shape[0]) # 2 == num of files

#avg_sub2.to_csv("atmacup3_sample_submission_avg2.csv", index = False)
#avg_sub2.head()